Khai báo các thư viện cần thiết

In [14]:
import os
import numpy as np

Khai báo đường dẫn tệp dữ liệu

In [15]:
data_path = '../../data/'

train_images_path = os.path.join(data_path, 'train-images-idx3-ubyte.gz')
train_labels_path = os.path.join(data_path, 'train-labels-idx1-ubyte.gz')

test_images_path = os.path.join(data_path, 't10k-images-idx3-ubyte.gz')
test_labels_path = os.path.join(data_path, 't10k-labels-idx1-ubyte.gz')

Hàm lấy dữ liệu

In [16]:
def get_mnist_data(images_path, labels_path, num_images, shuffle=False, _is=True, image_size=28):
    """
    This shuffle param is active when .gz is downloaded at:
    - 'http://yann.lecun.com/exdb/mnist/'
    - This function return random num_images in 60000 or 10000
    """
    # read data
    import gzip # to decompress gz (zip) file
    # open file training to read training data
    f_images = gzip.open(images_path,'r')
    # skip 16 first bytes because these are not data, only header infor
    f_images.read(16)
    # general: read num_images data samples if this parameter is set;
    # if not, read all (60000 training or 10000 test)
    real_num = num_images if not shuffle else (60000 if _is else 10000)
    # read all data to buf_images (28x28xreal_num)
    buf_images = f_images.read(image_size * image_size * real_num)
    # images
    images = np.frombuffer(buf_images, dtype=np.uint8).astype(np.float32)
    images = images.reshape(real_num, image_size, image_size,)
    # Read labels
    f_labels = gzip.open(labels_path,'r')
    f_labels.read(8)
    labels = np.zeros((real_num)).astype(np.int64)
    # rearrange to correspond the images and labels
    for i in range(0, real_num):
        buf_labels = f_labels.read(1)
        labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)

    # shuffle to get random images data
    if shuffle is True:
        rand_id = np.random.randint(real_num, size=num_images)
        images = images[rand_id, :]
        labels = labels[rand_id,]
    # change images data to type of vector 28x28 dimentional
    images = images.reshape(num_images, image_size * image_size)
    return images, labels

Lấy dữ liệu train, test từ tệp dữ liệu

In [17]:
train_images, train_labels = get_mnist_data(
    train_images_path, train_labels_path, 60000)
test_images, test_labels = get_mnist_data(
    test_images_path, test_labels_path, 10000, _is=False)
print(train_images.shape, train_labels.shape)
print(test_images.shape, test_labels.shape)

C:\Users\dangngocquan\AppData\Local\Temp\ipykernel_1320\3785225137.py:28: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  labels[i] = np.frombuffer(buf_labels, dtype=np.uint8).astype(np.int64)


(60000, 784) (60000,)
(10000, 784) (10000,)


Chuẩn hóa dữ liệu

In [18]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_x = scaler.fit_transform(train_images)
test_x = scaler.fit_transform(test_images)

print(train_x.shape)
print(test_x.shape)

(60000, 784)
(10000, 784)


Giảm số chiều dữ liệu về 100 chiều

In [19]:
from sklearn.decomposition import PCA

pca = PCA(n_components=100)
pca.fit(train_images)
X_train = pca.transform(train_images)
X_test = pca.transform(test_images)

print(X_train.shape)
print(X_test.shape)

(60000, 100)
(10000, 100)


Thực hiện quá trình one-hot encoding

In [20]:
from keras.utils import to_categorical

y_train = to_categorical(train_labels, num_classes=10)
y_test = to_categorical(test_labels, num_classes=10)

Khởi tạo mô hình ANN

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(10, activation='sigmoid', input_shape=(100,)))
model.add(Dense(10, activation='softmax'))

g:\app\python\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Tổng quan cấu trúc mô hình

In [22]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_2 (Dense)                 │ (None, 10)             │         1,010 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 10)             │           110 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,120 (4.38 KB)

 Trainable params: 1,120 (4.38 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
model.compile(loss="categorical_crossentropy",
              optimizer="sgd",
              metrics = ['accuracy'])

Huấn luyện mô hình với tập dữ liệu train

In [24]:
model.fit(X_train, y_train, batch_size=100, epochs=10)

Epoch 1/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.2013 - loss: 2.2179
Epoch 2/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5325 - loss: 1.6460
Epoch 3/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.6751 - loss: 1.3544
Epoch 4/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.7271 - loss: 1.1642
Epoch 5/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 985us/step - accuracy: 0.7689 - loss: 1.0205
Epoch 6/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8014 - loss: 0.9135
Epoch 7/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8122 - loss: 0.8366
Epoch 8/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8233 - loss: 0.7809
Epoch 9/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8304 - loss: 0.7349
Epoch 10/10
600/600 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.8360 - loss: 0.6903


Sử dụng mô hình đã huấn luyện dự đoán tập dữ liệu test

In [25]:
y_pred = model.predict(X_test)
y_pred = y_pred.argmax(axis = 1)
print(y_pred)

313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step
[7 0 1 ... 4 8 6]


Tính độ chính xác của mô hình

In [26]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score

accuracy = accuracy_score(test_labels, y_pred)
print("Accuracy:", accuracy)

confusion_matrix = confusion_matrix(test_labels, y_pred)
print("Confusion Matrix:")
print(confusion_matrix)

precision = precision_score(test_labels, y_pred, average='macro')
print("Precision:", precision)

recall = recall_score(test_labels, y_pred, average='macro')
print("Recall:", recall)

Accuracy: 0.8384
Confusion Matrix:
[[ 904    0    9   10    4   12   26   13    2    0]
 [   0 1101    7    4    0    2    5    3   11    2]
 [  40   15  809   69   12    5   27   23   25    7]
 [  20   14   32  827    0   46    6   15   40   10]
 [   4    1    7    0  810    1   22   11    2  124]
 [  25    5   12   98   21  628   30   10   57    6]
 [  12    4   25    0    7   38  859    7    6    0]
 [   9   22   38   10    3    0    4  871    2   69]
 [  17   30    6   49   19   66   21    7  744   15]
 [  10   12    6   13   73    2    5   46   11  831]]
Precision: 0.8366015105358378
Recall: 0.8355486819804469
